Using PyTorch as compute engine and mpi4py for communication, Heat implements a number of machine learning algorithms that are optimized for memory-distributed data volumes. This allows you to tackle datasets that are too large for single-node (or worse, single-GPU) processing. 

As opposed to task-parallel frameworks, Heat takes a data-parallel approach, meaning that each "worker" or MPI process performs the same tasks on different slices of the data, and communication between processes is performed transparently when the task requires it. 

In other words: you don't have to worry about optimizing data chunk sizes; you don't have to make sure your research problem is embarassingly parallel, or artificially make your dataset smaller so your RAM is sufficient. You do have to make sure that you have sufficient **overall** RAM to run your global task (e.g. number of nodes / GPUs).

The following shows a few examples. We'll use "small" datasets here as each of us only has access to one node only.

# clustering


# truncated SVD

# QR decomposition

# work in progress